# Zero-Shot Prompting

Zero-shot prompting refers to a technique in prompt engineering where you provide a model with a task without any prior examples. The model is expected to understand and generate a response or complete the task purely based on the given instruction.

In other words, the model is given "zero" prior training examples or demonstrations in the prompt and relies on its pre-trained knowledge to infer what is needed.

## References:
* [Wei et al. (2022)](https://arxiv.org/pdf/2109.01652.pdf): demonstrate how instruction tuning improves zero-shot learning 

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fzero_shot.ipynb)



In [ ]:
##
## ZERO SHOT PROMPTING
##

from _pipeline import create_payload, model_req

#### (1) Adjust the inbounding  Prompt, simulating inbounding requests from users or other systems
MESSAGE = "What is 984 * log(2)"

#### (2) Adjust the Prompt Engineering Technique to be applied, simulating Workflow Templates
PROMPT = MESSAGE 

#### (3) Configure the Model request, simulating Workflow Orchestration
# Documentation: https://github.com/ollama/ollama/blob/main/docs/api.md
payload = create_payload(target="ollama",
                         model="llama3.2:latest", 
                         prompt=PROMPT, 
                         temperature=1.0, 
                         num_ctx=100, 
                         num_predict=100)

### YOU DONT NEED TO CONFIGURE ANYTHING ELSE FROM THIS POINT
# Send out to the model
time, response = model_req(payload=payload)
print(response)
if time: print(f'Time taken: {time}s')

To calculate this, we need to know that the logarithm of 2 (base 10) is approximately 0.301.

So,

984 * log(2) ≈ 984 * 0.301
≈ 295.584
Time taken: 5.399s


---

## How to improve it?

* **Use Clear and Concise Instructions**: Be specific about the task and desired format.
    * Bad Prompt: “Summarize this.”
    * Good Prompt: “Summarize this paragraph in one sentence.”
* **Add Context**: Providing background can help the model interpret ambiguous prompts better.
* **Specify Output Format**: If a particular structure is needed, describe it in the instruction.

In [2]:
from _pipeline import create_payload, model_req

#### (1) Adjust the inbounding Prompt, simulating inbounding requests from users or other systems
JOB_DESCRIPTION = """
We are hiring a Data Scientist with expertise in Python, Machine Learning, and SQL.
The candidate must have at least 3 years of experience and a background in statistical modeling.
Experience with cloud platforms (AWS, GCP) is a plus.
"""

RESUME = """
John Doe is a Data Scientist with 4 years of experience. Skilled in Python, Machine Learning,
and SQL. He has worked on predictive modeling projects and statistical analysis.
Familiar with AWS but lacks GCP experience.
"""

#### (2) Adjust the Zero-Shot Prompt
ZERO_SHOT_PROMPT = f"""
You are an AI hiring assistant. Analyze the following job description and candidate resume. 
Compare the qualifications and provide a structured evaluation with a weighted match score.

---

### **Evaluation Criteria:**
- **Required skills missing:** -10% per missing skill.
- **Preferred skills missing:** -5% per missing skill.
- **Exceeds required experience:** +2% per extra year (max +5%).
- **Final score is capped at 100%**.

---

**Job Description:**  
{JOB_DESCRIPTION}  

**Candidate Resume:**  
{RESUME}  

---

### **Instructions:**  
1️⃣ Extract key skills, experience, and certifications from both documents.  
2️⃣ Identify matches and gaps between the job description and the candidate's qualifications.  
3️⃣ Apply the weighted scoring system to determine the final match score.  
4️⃣ Provide a **concise hiring recommendation** based on the score:  
   - **90%+ → Strong Fit (Highly Recommended)**  
   - **75%-89% → Good Fit (Recommended with Minor Improvements)**  
   - **Below 75% → Needs Improvement (Not Recommended)**  

---

### **Output Format:**  
✔ **Evaluation Summary:** Key matches and missing skills.  
✔ **Match Score:** XX% (numerical only).  
✔ **Hiring Recommendation:** Decision based on final match score.  

Provide the structured response exactly as specified. Do not include explanations beyond the requested sections.
"""

PROMPT = ZERO_SHOT_PROMPT

#### (3) Configure the Model Request
payload = create_payload(target="ollama",
                         model="mistral",  
                         prompt=PROMPT, 
                         temperature=0.7,  
                         num_ctx=768,  # Optimized for resume screening
                         num_predict=350,  # Increased to ensure complete structured output
                         stream=True)  # Enable streaming for better performance

### YOU DON'T NEED TO CONFIGURE ANYTHING ELSE FROM THIS POINT
# Send out to the model
time, response = model_req(payload=payload)
print(response)
if time: print(f'Time taken: {time}s')

{'model': 'mistral', 'prompt': "\nYou are an AI hiring assistant. Analyze the following job description and candidate resume. \nCompare the qualifications and provide a structured evaluation with a weighted match score.\n\n---\n\n### **Evaluation Criteria:**\n- **Required skills missing:** -10% per missing skill.\n- **Preferred skills missing:** -5% per missing skill.\n- **Exceeds required experience:** +2% per extra year (max +5%).\n- **Final score is capped at 100%**.\n\n---\n\n**Job Description:**  \n\nWe are hiring a Data Scientist with expertise in Python, Machine Learning, and SQL.\nThe candidate must have at least 3 years of experience and a background in statistical modeling.\nExperience with cloud platforms (AWS, GCP) is a plus.\n  \n\n**Candidate Resume:**  \n\nJohn Doe is a Data Scientist with 4 years of experience. Skilled in Python, Machine Learning,\nand SQL. He has worked on predictive modeling projects and statistical analysis.\nFamiliar with AWS but lacks GCP experienc